In [95]:
import pandas as pd
import json

class apiData():
    '''
    Holds api data fetched from the Wiener Linien API
    Can clean, format and return datasets for specific lines
    '''

    def __init__(self, api_file) -> None:
        '''
        api_file: pandas compatible csv file containing response data from the Wiener Linien API (may be gzipped)
        '''
        self.df = pd.read_csv(api_file)

    def getAvailable(self, filter = True):
        '''
        returns a dataframe with available lines and directions
        filter (bool): if set to true, filters the unique entries for special announcements
        '''
        available = self.df.groupby(['station', 'line', 'towards']).size().rename('count').reset_index()

        if filter:
            available = available.loc[~available['towards'].str.match('(.*\*)|(.*_)|.*(FFP2-MASKEN)|(.*\!)|.*NÄCHSTER|.*FAHRTBEHINDERUNG')]

        return available.sort_values('count', ascending=False)


    def getLineDir(self, station, line, direction):
        '''
        TODO
        returns a formatted dataframe containing the defined line and direction
        '''
        temp_df = self.df.loc[(self.df['station'] == station) & (self.df['line'] == line) & (self.df['towards'] == direction)]
        temp_df = self.clean(temp_df)
        temp_df = self.format(temp_df)

        return temp_df

    def clean(self, sub_df, filter_regex = True, filter_dupes = True):
        '''
        TODO
        Input is an already filtered dataframe for one station/line/direction combination.
        cleans up a dataframe (remove bad responses) and returns it
        In case there are duplicates (two entries at the exaxt same time) both entries are dropped

        filter_regex (bool): turns on/off regex filtering
        filter_dupes (bool): turns on/off duplicate filtering
        '''
        if filter_regex:
            sub_df = sub_df.loc[sub_df['countdown'].str.match('\[[0-9, ]*\]')]
        
        if filter_dupes:
            sub_df = sub_df.drop_duplicates(subset='time', keep=False)

        return sub_df

    def format(self, sub_df):
        '''
        TODO
        returns a formatted dataframe ready for analysis
        '''
        # first, convert the countdown list into columns
        cntdwn = sub_df['countdown']
        cntdwn = pd.DataFrame([json.loads(x) for x in cntdwn])
        cntdwn = cntdwn.set_index(sub_df.index)

        sub_df = pd.concat([sub_df, cntdwn], axis=1).drop(columns='countdown')

        return sub_df

In [96]:
pd.options.display.max_rows = 25
data = apiData('./data/scrape.csv.gz')
data.getAvailable()

,station,line,towards,count
210,Längenfeldgasse,U6,SIEBENHIRTEN,55493
53,Längenfeldgasse,U4,HÜTTELDORF,55149
150,Längenfeldgasse,U6,FLORIDSDORF,55078
1,Flurschützstraße / Längenfeldgasse,63A,Gesundheitszentrum Süd,55030
0,Flurschützstraße / Längenfeldgasse,63A,Am Rosenhügel,54890
...,...,...,...,...
13,"Flurschützstraße, Längenfeldgasse",62,Quartier Belvedere S,63
89,Längenfeldgasse,U4,MEIDLING HAUPT,46
148,Längenfeldgasse,U6,BHF. MEIDLING,42
137,Längenfeldgasse,U4,SCHWEDENPLATZ,17


In [97]:
data.getLineDir('Längenfeldgasse', 'U4', 'HÜTTELDORF')

,station,line,towards,time,0,1,2,3,4,5,...,12,13,14,15,16,17,18,19,20,21
7,Längenfeldgasse,U4,HÜTTELDORF,1.637752e+09,1,6,9.0,14.0,19.0,24.0,...,59.0,64.0,69.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,Längenfeldgasse,U4,HÜTTELDORF,1.637752e+09,4,8,13.0,18.0,23.0,28.0,...,63.0,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,Längenfeldgasse,U4,HÜTTELDORF,1.637752e+09,4,8,12.0,17.0,22.0,27.0,...,62.0,67.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,Längenfeldgasse,U4,HÜTTELDORF,1.637752e+09,4,8,12.0,17.0,22.0,27.0,...,62.0,67.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,Längenfeldgasse,U4,HÜTTELDORF,1.637752e+09,4,8,12.0,17.0,22.0,27.0,...,62.0,67.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711234,Längenfeldgasse,U4,HÜTTELDORF,1.638530e+09,2,7,10.0,15.0,20.0,25.0,...,60.0,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
711246,Längenfeldgasse,U4,HÜTTELDORF,1.638530e+09,2,7,10.0,15.0,20.0,25.0,...,60.0,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
711258,Längenfeldgasse,U4,HÜTTELDORF,1.638530e+09,2,7,10.0,15.0,20.0,25.0,...,60.0,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
711270,Längenfeldgasse,U4,HÜTTELDORF,1.638530e+09,1,7,10.0,15.0,20.0,25.0,...,60.0,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
av.loc[~av['towards'].str.match('(.*\*)|(.*_)|.*(FFP2-MASKEN)|(.*\!)|.*NÄCHSTER|.*FAHRBEHINDERUNG')]

,station,line,towards,count
0,Flurschützstraße / Längenfeldgasse,63A,Am Rosenhügel,54890
1,Flurschützstraße / Längenfeldgasse,63A,Gesundheitszentrum Süd,55030
2,Flurschützstraße / Längenfeldgasse,63A,Meidling Hauptstraße U,64
3,Flurschützstraße / Längenfeldgasse,63A,Niederhofstraße U,232
4,"Flurschützstraße, Längenfeldgasse",18,"Burggasse, Stadthalle U",622
5,"Flurschützstraße, Längenfeldgasse",18,"Meidling S, Dörfelstraße",745
6,"Flurschützstraße, Längenfeldgasse",18,Schlachthausgasse U,767
7,"Flurschützstraße, Längenfeldgasse",6,Geiereckstraße,596
8,"Flurschützstraße, Längenfeldgasse",6,"Meidling S, Dörfelstraße",302
9,"Flurschützstraße, Längenfeldgasse",62,Hermesstraße,1701


In [20]:

data.df.groupby(['station', 'line', 'towards']).size().rename('count').reset_index()
#data.df.value_counts(subset=['station', 'line', 'towards'])

,station,line,towards,count
0,Flurschützstraße / Längenfeldgasse,63A,Am Rosenhügel,54890
1,Flurschützstraße / Längenfeldgasse,63A,Gesundheitszentrum Süd,55030
2,Flurschützstraße / Längenfeldgasse,63A,Meidling Hauptstraße U,64
3,Flurschützstraße / Längenfeldgasse,63A,Niederhofstraße U,232
4,"Flurschützstraße, Längenfeldgasse",18,"Burggasse, Stadthalle U",622
5,"Flurschützstraße, Längenfeldgasse",18,"Meidling S, Dörfelstraße",745
6,"Flurschützstraße, Längenfeldgasse",18,Schlachthausgasse U,767
7,"Flurschützstraße, Längenfeldgasse",6,Geiereckstraße,596
8,"Flurschützstraße, Längenfeldgasse",6,"Meidling S, Dörfelstraße",302
9,"Flurschützstraße, Längenfeldgasse",62,Hermesstraße,1701
